In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE57225'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Intra-individual genome expression analysis reveals a specific molecular signature of psoriasis and eczema"
!Series_summary	"Comparing molecular signatures of psoriasis and eczema in patients co-affected by both diseases provides a comprehensive understanding of disease pathogenesis as well as a diagnostic tool to differentiate these widespread inflammatory skin diseases."
!Series_overall_design	"In patients affected by both psoriasis and non-atopic or atopic eczema simultaneously (n=24), whole genome expression arrays of psoriasis, eczema, and non-involved skin were performed"
!Series_overall_design	"Arrays MQ_35 and MQ_41 did not pass quality control and thus were not normalized and were excluded from this Series."
Sample Characteristics Dictionary:
{0: ['individual: EP', 'individual: KK', 'individual: KP', 'individual: SI', 'individual: MM', 'individual: ZA', 'individual: LA', 'individual: MOF', 'individual: Mattes', 'individual: SS2', 'individ

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 1
age_row = 4
gender_row = 3

def convert_trait(value):
    value = value.split(':')[1].strip()
    if value == 'eczema':
        return 1
    elif value == 'psoriasis' or value == 'control (non-involved)':
        return 0
    else:
        return None

def convert_age(value):
    try:
        return int(value.split(':')[1].strip().replace('y', ''))
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[1].strip()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE57225', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)

# If the trait_row is not None, proceed with clinical feature extraction.
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Eczema/trait_data/GSE57225.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1377461': [0, 48, 1], 'GSM1377462': [1, 48, 1], 'GSM1377463': [0, 40, 1], 'GSM1377464': [1, 40, 1], 'GSM1377465': [0, 65, 1], 'GSM1377466': [0, 65, 1], 'GSM1377467': [1, 65, 1], 'GSM1377468': [0, 35, 0], 'GSM1377469': [1, 35, 0], 'GSM1377470': [0, 27, 0], 'GSM1377471': [1, 27, 0], 'GSM1377472': [0, 65, 0], 'GSM1377473': [1, 65, 0], 'GSM1377474': [0, 72, 1], 'GSM1377475': [0, 72, 1], 'GSM1377476': [1, 72, 1], 'GSM1377477': [0, 33, 0], 'GSM1377478': [0, 33, 0], 'GSM1377479': [1, 33, 0], 'GSM1377480': [0, 48, 1], 'GSM1377481': [0, 48, 1], 'GSM1377482': [1, 48, 1], 'GSM1377483': [0, 58, 0], 'GSM1377484': [0, 58, 0], 'GSM1377485': [1, 58, 0], 'GSM1377486': [0, 65, 0], 'GSM1377487': [0, 65, 0], 'GSM1377488': [1, 65, 0], 'GSM1377489': [0, 56, 0], 'GSM1377490': [0, 56, 0], 'GSM1377491': [1, 56, 0], 'GSM1377492': [0, 46, 0], 'GSM1377493': [0, 46, 0], 'GSM1377494': [1, 46, 0], 'GSM1377495': [0, 55, 0], 'GSM1377496': [1, 55, 0], 'GSM1377497': [0, 46, 0], 'GSM1377498': [0, 46, 0], 'GSM1377499

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315469', 'A_19_P00315473',
       'A_19_P00315482', 'A_19_P00315490', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315499', 'A_19_P00315502', 'A_19_P00315504', 'A_19_P00315506',
       'A_19_P00315508', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315523',
       'A_19_P00315524', 'A_19_P00315526', 'A_19_P00315527', 'A_19_P00315528'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert probe IDs to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Eczema/gene_data/GSE57225.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Eczema')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE57225', './preprocessed/Eczema/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Eczema/GSE57225.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Eczema', the least common label is '1.0' with 23 occurrences. This represents 37.10% of the dataset.
The distribution of the feature 'Eczema' in this dataset is fine.

Quartiles for 'Age':
  25%: 36.25
  50% (Median): 46.0
  75%: 56.0
Min: 20.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 19 occurrences. This represents 30.65% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

